# Data Cleaning
Script for cleaning up data in quotes excel sheet for VADER input

Each row should contain:
* Text id (from 'full' tab)
* Text name ('full')
* Quotes
 * All quotes in 'full' tab
 * All quotes from the same file in the same row
 * Quotes separated by newline (merge quotes)
* Non-quotes ('non_quoted_text', copied as is)
* Speaker ('full')
 * Merged and newline separated, as with Quotes
* Verb ('full')
 * Merged and newline separated, as with Quotes

Output CSV header names: text_id, text_name, quotes, non_quotes, speakers, verbs

In [ ]:
# run this code if connecting to a Google drive
from google.colab import drive

drive.mount('/content/drive')

In [2]:
import pandas as pd
import re

## Important!! Remember to replace file paths with the correct ones when running this code locally

In [13]:
# extract relevant columns from both excel sheets

# replace this with relevant file path
fp = '/content/drive/My Drive/evaluation_quotes/quotes.xlsx'

full_df = pd.read_excel(fp, sheet_name = 'full', usecols = ["text_id", "text_name", "quote", "speaker", "verb"])

non_quotes_df = pd.read_excel(fp, sheet_name = 'non_quoted_text')

In [ ]:
# check number of articles in the non_quoted_text sheet
len(non_quotes_df)

In [ ]:
# check number of articles in the full sheet
unique_ids = full_df['text_id'].nunique()

unique_names = full_df['text_name'].nunique()

print("number of unique ids: ", unique_ids)
print("number of unique names: ", unique_names)

In [ ]:
# merge quotes, speakes, verbs separated by newline
merged_df = full_df.groupby(['text_id', 'text_name']).agg({
    'quote': lambda x: '\n'.join(x),
    'speaker': lambda x: '\n'.join(map(str, x)),
    'verb': lambda x: '\n'.join(map(str, x))
}).reset_index()

merged_df.head()

In [17]:
# clean up speaker and verb columns by replacing 'nan' with newlines
for index, row in merged_df.iterrows():
  speakers = re.sub(r'nan', '\n', merged_df.loc[index, 'speaker'])
  verbs = re.sub(r'nan', '\n', merged_df.loc[index, 'verb'])

  merged_df.loc[index, 'speaker'] = speakers
  merged_df.loc[index, 'verb'] = verbs

In [ ]:
# merge the updated full dataframe with the non_quotes dataframe
# with the new headers text_id, text_name, quote, speaker, verb, non_quoted_text
output_df = pd.merge(merged_df, non_quotes_df, on=['text_id', 'text_name'])

output_df.head()

## Remember to replace the output file path with your own specifications

In [19]:
# write dataframe to excel sheet
# replace this file path with the correct/relevant file path
output = '/content/drive/My Drive/evaluation_quotes/quotes_input.xlsx'

output_df.to_excel(output, index=False)